In [1]:
from py_plink import read_plink
import numpy as np
import pandas as pd
from collections import OrderedDict as odict
from math import isnan
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import timeit
from aco import pre_processing, post_processing
import multiprocessing as mp

In [11]:
class single_assoc:
    def __init__(self, bim, fam, bed, cases_i, controls_i):
        self.bim = bim
        self.bed = bed
        self.fam = fam
        self.cases_i = cases_i
        self.controls_i = controls_i
        
        self.n_individuals = self.fam.shape[0]
        self.n_snps = self.bim.shape[0]
        self.b_snp = bed[298170].compute()
        
        
        
    def chi_sq(self, snp):
        snp_mat = self.bed[snp].compute()
        stat = 0
        t_cases = np.zeros(4)
        t_controls = np.zeros(4)
        p = 100

        for i in self.cases_i:
            if(isnan(snp_mat[i])):
                snp_mat[i] = 3
            t_cases[int(snp_mat[i])] += 1

        for i in self.controls_i:
            if(isnan(snp_mat[i])):
                snp_mat[i] = 3
            t_controls[int(snp_mat[i])] += 1
        if(t_cases[0] != 0 and t_cases[1] != 0 and t_cases[2] != 0 and t_controls[0] != 0 and t_controls[1] != 0 and t_controls[2] != 0): 
            table = [[t_cases[0], t_cases[1], t_cases[2]],
                    [t_controls[0], t_controls[1], t_controls[2]]]

            stat, p, dof, expected = chi2_contingency(table)
            print(dof)
        return stat
     
    def chi_sq_omnibus(self, snp1):
        snp1_mat = self.b_snp
        snp2_mat = self.bed[snp1].compute()
        
        t_cases = np.zeros((4,4))
        t_controls = np.zeros((4,4))
        table_i = []
        table_j = []
        
        for i in self.cases_i:
            if(isnan(snp1_mat[i])):
                snp1_mat[i] = 3
                
            if(isnan(snp2_mat[i])):
                snp2_mat[i] = 3  
                
            t_cases[int(snp1_mat[i])][int(snp2_mat[i])] += 1
            
            
        for i in self.controls_i:
            
            if(isnan(snp1_mat[i])):
                snp1_mat[i] = 3
                
            if(isnan(snp2_mat[i])):
                snp2_mat[i] = 3
                
            t_controls[int(snp1_mat[i])][int(snp2_mat[i])] += 1
            
        
        table_i = t_cases[:3,:3]
        table_i = table_i.flatten()
        non_zeros = np.nonzero(table_i)
        table_i = table_i[non_zeros]
        
        table_j = t_controls[:3,:3]
        table_j = table_j.flatten()
        table_j = table_j[non_zeros]
        
        table = [table_i, table_j]
        stat, p, dof, expected = chi2_contingency(table)
        stop = timeit.default_timer()
        
        
        
        return p 
    
    def run(self):
        start = timeit.default_timer()
        #pool = mp.Pool(mp.cpu_count())
        p_values = []
        for i in range(self.n_snps):
            p_values.append(self.chi_sq_omnibus(i))
            print(i)
            
        #p_values = pool.map(self.chi_sq_omnibus, [row for row in range(self.n_snps)])
        top_snps = np.argsort(p_values)[:20]
        stop = timeit.default_timer()
        #pool.close()
        for i in top_snps:
            print("SNP: ", bim.loc[i, 'snp'], "P-value: ", p_values[i])
            
        print('Time: ', stop - start)
            
        return p_values
                
            
    

In [12]:
plink_fn = '/Users/raouldias/Desktop/Extend/extend_csp_data_annon'
pheno_fn = '/Users/raouldias/Desktop/Extend/extend_phenotype.txt'
bim, fam, bed, cases_i, controls_i = pre_processing(plink_fn, pheno_fn, nbant, nbt, evaporation_rate, init_val, total_fitness_evals).run()
single_assoc_test = single_assoc(bim, fam, bed, cases_i, controls_i)
p_values = single_assoc_test.run()
